In [1]:
import sys
from google.colab import drive

drive.mount('/content/drive')
sys.path.append(r"/content/drive/MyDrive/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from loader_util.preprocessing import ImageToArrayPreprocessor
from loader_util.preprocessing import SimplePreProcessor
from loader_util.preprocessing import MeanPreprocessor
from loader_util.preprocessing import CropPreprocessor
from loader_util.io import HDF5DatasetGenerator
from loader_util.utils.ranked import rank5_accuracy
from tensorflow.keras.models import load_model, Model
import numpy as np
import json

In [3]:
DATASET_MEAN_PATH = r"/content/drive/MyDrive/Colab Notebooks/pyimagesearch/bibilio2/chap10/dogs_cats_mean.json"
MODEL_PATH = r"/content/drive/MyDrive/Colab Notebooks/pyimagesearch/bibilio2/chap10/saved_model"
TEST_HDF5_PATH = r"/content/drive/MyDrive/ImageDataset/book2/kaggle_dogs_vs_cats/hdf5/test.hdf5"

# read the dataset mean json
means = json.loads(open(DATASET_MEAN_PATH, mode="r").read())

# init the preprocessors
sp = SimplePreProcessor(227, 227)
mp = MeanPreprocessor(rmean=means['R'], gmean=means['G'], bmean=means['G'])
cp = CropPreprocessor(227, 227)
iap = ImageToArrayPreprocessor()

# load the saved model
print(f"[INFO] loading the saved model......")
model = load_model(MODEL_PATH)  # type: Model
model.summary()

[INFO] loading the saved model......
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 57, 57, 96)        34944     
                                                                 
 activation_8 (Activation)   (None, 57, 57, 96)        0         
                                                                 
 batch_normalization_7 (Batc  (None, 57, 57, 96)       384       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 28, 28, 96)       0         
 2D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 28, 28, 96)        0         
                                                                 
 conv2d_6 (Conv2D

In [4]:
print(f"[INFO] predicting on test data with no crops......")
test_gen = HDF5DatasetGenerator(db_path=TEST_HDF5_PATH,
                                batch_size=64,
                                preprocessors=[sp, mp, iap],
                                classes=2)
predictions = model.predict_generator(test_gen.generator(),
                                      steps=test_gen.num_images // 64,
                                      max_queue_size=10)

[INFO] predicting on test data with no crops......


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  


In [5]:
# compute the rank1 and rank5 accuracy
rank1, _ = rank5_accuracy(preds=predictions,
                          labels=test_gen.db['labels'])
print(f"[INFO] rank1 accuracy: {rank1 * 100}......")
test_gen.close()

[INFO] rank1 accuracy: 89.74358974358975......


In [6]:
# reinit the testing set generator this time exclusing the SimplePreprocessor
test_gen = HDF5DatasetGenerator(db_path=TEST_HDF5_PATH,
                                batch_size=64,
                                preprocessors=[mp],
                                classes=2)

In [7]:
predictions = []
for i , (images, labels) in enumerate(test_gen.generator(passes=1)):
    for image in images:
        crops = cp.proprocess(image)
        crops = np.array([iap.preprocess(c) for c in crops], dtype="float32")

        pred = model.predict(crops)
        predictions.append(pred.mean(axis=0))

1/1 [==============================] - 0s 20ms/step


In [8]:
# compute the rank1 and rank5 accuracy
rank1, _ = rank5_accuracy(preds=predictions,
                          labels=test_gen.db['labels'])
print(f"[INFO] rank1 accuracy: {rank1 * 100}......")
test_gen.close()

[INFO] rank1 accuracy: 91.03999999999999......
